In [44]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import math, decimal
import requests
dec = decimal.Decimal
import hvplot.pandas
import krakenex
from pykrakenapi import KrakenAPI
api = krakenex.API()
k = KrakenAPI(api)
import json
FearGreedUrl = requests.get('https://api.alternative.me/fng/?limit=2').json()

In [45]:
from pathlib import Path
csvpath1 = Path("./full_moon.csv")

In [46]:
full_moon = pd.read_csv(csvpath1, parse_dates=True, index_col=' Date', infer_datetime_format=True)
full_moon.reset_index(inplace=True)
full_moon = full_moon.rename(columns = {' Date':'Full Moon'})
full_moon.drop(['Day',' Time'], axis=1, inplace=True)

In [47]:
full_moon['New Moon'] = full_moon['Full Moon'] + timedelta(days=14)
full_moon

,Full Moon,New Moon
0,1900-01-15,1900-01-29
1,1900-02-14,1900-02-28
2,1900-03-16,1900-03-30
3,1900-04-15,1900-04-29
4,1900-05-14,1900-05-28
...,...,...
1863,2050-09-01,2050-09-15
1864,2050-09-30,2050-10-14
1865,2050-10-30,2050-11-13
1866,2050-11-28,2050-12-12


In [48]:
ohlc = k.get_ohlc_data('BTCUSD', interval=1440, ascending = True)
Price = ohlc[0]['Price'] = ohlc[0]['close'].rolling(1).mean()
ohlc[0].drop(['high', 'low', 'time', 'open', 'close', 'vwap', 'count', 'volume'], axis = 1).reset_index()

,dtime,Price
0,2020-05-08,9809.5
1,2020-05-09,9539.7
2,2020-05-10,8716.5
3,2020-05-11,8570.8
4,2020-05-12,8823.7
...,...,...
715,2022-04-23,39435.6
716,2022-04-24,39467.9
717,2022-04-25,40438.5
718,2022-04-26,38121.7


In [49]:
# Create a Moon Data table with the BTC Price data for each Full and New Moon date

full_moon = full_moon.join(ohlc[0]['Price'], on = 'Full Moon', how = 'inner')
full_moon['Full Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

full_moon = full_moon.join(ohlc[0]['Price'], on = 'New Moon', how = 'inner')
full_moon['New Moon Price'] = full_moon['Price']
full_moon = full_moon.drop(columns=['Price']).reset_index()

moon_data = full_moon.drop(columns=['index','level_0'])

moon_data.tail()


,Full Moon,New Moon,Full Moon Price,New Moon Price
18,2021-11-19,2021-12-03,58099.6,53678.6
19,2021-12-19,2022-01-02,46688.4,47304.6
20,2022-01-18,2022-02-01,42383.0,38719.9
21,2022-02-16,2022-03-02,43873.1,43922.5
22,2022-03-18,2022-04-01,41775.1,46312.0


In [55]:
# create new column that is the difference between the Full Moon and New Moon prices
moon_data['Price Difference'] = moon_data['New Moon Price'] - moon_data['Full Moon Price']
moon_data.tail()

# create new column that is the percentage difference between the Full Moon and New Moon prices
moon_data['Percentage Difference'] = (moon_data['New Moon Price'] - moon_data['Full Moon Price']) / moon_data['Full Moon Price']
# sum the price difference column 
sum(moon_data['Price Difference'])

'''things to accomplish tomorrow. 
1. create new column that shows true if the price difference is positive, false if negative
2. create new column that shows the difference between when first bought and where the price is now (this is the control variable)
3. create new column that shows the difference between column created in step 3. and the price sum of the price difference column (this is the dependent variable)
4. plot the dependent variable against the control variable

5. create new column for the fear and greed data
7. create column for the all time high and low
8. calculate correlation between moon phases and all time high and low then plot 
9. calculate correlation between moon phases and fear and greed data then plot

'''


def result(x, y):
    if x - y > 0:
        return True
    return False

moon_data['Result'] = moon_data.apply(lambda row: result(row['New Moon Price'], row['Full Moon Price']), axis=1)
moon_data.tail()

,Full Moon,New Moon,Full Moon Price,New Moon Price,Price Difference,Percentage Difference,Result
18,2021-11-19,2021-12-03,58099.6,53678.6,-4421.0,-0.076093,False
19,2021-12-19,2022-01-02,46688.4,47304.6,616.2,0.013198,True
20,2022-01-18,2022-02-01,42383.0,38719.9,-3663.1,-0.086429,False
21,2022-02-16,2022-03-02,43873.1,43922.5,49.4,0.001126,True
22,2022-03-18,2022-04-01,41775.1,46312.0,4536.9,0.108603,True


In [57]:
#7. create column for the all time high and low
# moon_data['All Time High'] = Price.max()
# moon_data


,Full Moon,New Moon,Full Moon Price,New Moon Price,Price Difference,Percentage Difference,Result,All Time High
0,2020-06-05,2020-06-19,9617.7,9302.1,-315.6,-0.032814,False,67559.8
1,2020-07-05,2020-07-19,9083.8,9213.7,129.9,0.014300,True,67559.8
2,2020-08-03,2020-08-17,11237.3,12301.7,1064.4,0.094720,True,67559.8
3,2020-09-02,2020-09-16,11396.9,10951.1,-445.8,-0.039116,False,67559.8
4,2020-10-01,2020-10-15,10616.1,11501.0,884.9,0.083355,True,67559.8
5,2020-10-31,2020-11-14,13809.1,16080.1,2271.0,0.164457,True,67559.8
6,2020-11-30,2020-12-14,19708.1,19281.1,-427.0,-0.021666,False,67559.8
7,2020-12-30,2021-01-13,28879.9,37390.1,8510.2,0.294676,True,67559.8
8,2021-01-28,2021-02-11,33430.4,47962.8,14532.4,0.434706,True,67559.8
9,2021-02-27,2021-03-13,46170.2,61174.9,15004.7,0.324987,True,67559.8


In [51]:
# create hvplot figures to then overaly 
glyph_1 = ohlc[0]['Price'].hvplot.line( 
    'dtime', 'Price',
    color='purple',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_2 = moon_data.hvplot.scatter(
    x = 'Full Moon',
    y = 'Full Moon Price',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)

glyph_3 = moon_data.hvplot.scatter(
    x = 'New Moon',
    y = 'New Moon Price',
    xlabel='Date',
    ylabel='BTC Price',
    title='BTC Price over Time'
)


In [52]:
#overaly the figures created above 
glyph_1*glyph_2*glyph_3

:Overlay
   .Curve.Price :Curve   [dtime]   (Price)
   .Scatter.I   :Scatter   [Full Moon]   (Full Moon Price)
   .Scatter.II  :Scatter   [New Moon]   (New Moon Price)

In [53]:
print(json.dumps(FearGreedUrl["data"][0], indent=4))

{
    "value": "21",
    "value_classification": "Extreme Fear",
    "timestamp": "1651017600",
    "time_until_update": "80082"
}
